In [168]:
import pandas as pd
import numpy as np
import re

import sys

sys.path.append("../src/biblical_scripts/pipelines/data_science")
sys.path.append("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/src")

from AuthorshipAttribution.AuthAttLib import *

from tqdm import tqdm

import logging
logging.basicConfig(level=logging.DEBUG)


In [169]:
path = "/Users/kipnisal/DS/BiblicalScripts/bib-scripts"

In [23]:
from kedro.framework.context import load_context
cnx = load_context('/Users/kipnisal/DS/BiblicalScripts/bib-scripts')

INFO:anyconfig:Loading: /Users/kipnisal/DS/BiblicalScripts/bib-scripts/pyproject.toml


In [170]:
from kedro.config import ConfigLoader

conf_paths = [path +'/'+ "conf/base",path +'/'+ "conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")

conf_paths = [path + '/' + "conf/base",path + '/' + "conf/local"]
conf_loader = ConfigLoader(conf_paths)
parameters = conf_loader.get("parameters*", "parameters*/**")


INFO:anyconfig:Loading: /Users/kipnisal/DS/BiblicalScripts/bib-scripts/conf/base/catalog.yml
INFO:anyconfig:Loading: /Users/kipnisal/DS/BiblicalScripts/bib-scripts/conf/base/parameters.yml


# Text Processing

In [233]:
data = pd.read_csv(path + '/' + 'data/02_intermediate/oshd_parsed.csv')
from biblical_scripts.pipelines.data_engineering.TextProcessing import TextProcessing
tp = TextProcessing(extract_prefix=True, ng_range=(2,2))
data_proc = tp.proc_ng(data)
data_proc

INFO:root:Extracting prefixes and suffixes
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,author,chapter,verse,feature,token_id
0,"1, 1 Chron 10",1Chr.10,1Chr.10.1,"(c, 6430)","(0, 0)"
1,"1, 1 Chron 10",1Chr.10,1Chr.10.1,"(6430, 3898)","(0, 1)"
2,"1, 1 Chron 10",1Chr.10,1Chr.10.1,"(3898, b)","(1, 2)"
3,"1, 1 Chron 10",1Chr.10,1Chr.10.1,"(b, 3478)","(2, 2)"
4,"1, 1 Chron 10",1Chr.10,1Chr.10.1,"(3478, c)","(2, 3)"
...,...,...,...,...,...
91720,Prov,Prov.31,Prov.31.9,"(6664, 1777)","(63196, 63197)"
91721,Prov,Prov.31,Prov.31.9,"(1777, c)","(63197, 63197)"
91722,Prov,Prov.31,Prov.31.9,"(c, 6041)","(63197, 63198)"
91723,Prov,Prov.31,Prov.31.9,"(6041, c)","(63198, 63199)"


# Conversion

In [257]:
from biblical_scripts.extras.Convert import Convert
convert = Convert(data)

In [263]:
convert.ng2term(ngram = ('120', '1456'))

['אָדָ֣ם', 'גֵּהָ֑ה']

# BS

# Models

In [ ]:
data_proc = pd.read_csv(path + '/' + 'data/04_feature/data_proc.csv')

In [243]:
import pandas as pd
import numpy as np
import logging
from biblical_scripts.pipelines.data_science.AuthorshipAttribution.AuthAttLib import AuthorshipAttributionDTM
from biblical_scripts.pipelines.data_science.AuthorshipAttribution.MultiDoc import CompareDocs

from biblical_scripts.pipelines.data_science.nodes import (build_vocab, compute_sim)

In [48]:
def compute_sim(data, vocabulary, params, known_authors) :
    ds = data.rename(columns = {'feature' : 'term', 'chapter' : 'doc_id'}).dropna()
    model = AuthorshipAttributionDTM(ds, params, vocab=list(vocabulary.feature.values))
    return model.compute_inter_similarity(LOO = True, wrt_authors=known_authors)

In [53]:
vocab = pd.read_csv(path + '/' + 'data/05_model_input/vocabulary.csv')
data = pd.read_csv(path + '/' + 'data/04_feature/data_proc.csv')

params = {}
[params.update(p) for p in parameters['HC']]

ds = data.rename(columns = {'feature' : 'term', 'chapter' : 'doc_id'}).dropna()
model = AuthorshipAttributionDTM(ds, **params, vocab=list(vocab.feature.values))